<a href="https://colab.research.google.com/github/cgN77/pclubML/blob/main/TweetStuff_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tweet Stuff


In [1]:
import pdb
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer
from os import getcwd
from nltk.stem import PorterStemmer
import re
from nltk.tokenize import word_tokenize

In [2]:
nltk.download('stopwords')
nltk.download('twitter_samples')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [16]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))
len(train_y)

8000

In [5]:
stemmer = PorterStemmer()
def stem_words(words):
    return [stemmer.stem(word) for word in words]
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
  # words = text.split()
  return [lemmatizer.lemmatize(word) for word in text]

Tweet processing

In [6]:
tw = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"

url_pattern = re.compile(r'https?://\S+')

def remove_urls(text):
    return url_pattern.sub('', text)

def process_tweet(tweet):
  tweet=tweet.lower()
  tweet=remove_urls(tweet)
  tweet = re.sub(r'[^\w\s]', '', tweet)
  tweet=re.sub(r'\d','',tweet)
  tweet=word_tokenize(tweet)
  stop_words = set(stopwords.words('english'))
  tweet = [word for word in tweet if word.lower() not in stop_words]
  tweet=stem_words(tweet)
  tweet=lemmatize_words(tweet)
  return tweet

tw=process_tweet(tw)
tw

['rt', 'twitter', 'chapagain', 'hello', 'great', 'day', 'good', 'morn']

Counting words

In [11]:
def count_tweets(result,tweets,ys):
  for y,tweet in zip(ys,tweets):
    for word in process_tweet(tweet):
      pair=(word,y)
      if pair in result:
        result[pair]+=1
      else:
        result[pair]=1
  return result


In [12]:
result = {}
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
count_tweets(result, tweets, ys)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}

In [21]:
freqs = count_tweets({}, train_x, train_y)
freqs

{('followfriday', 1.0): 23,
 ('france_int', 1.0): 1,
 ('pkuchli', 1.0): 1,
 ('milipol_pari', 1.0): 1,
 ('top', 1.0): 29,
 ('engag', 1.0): 7,
 ('member', 1.0): 14,
 ('commun', 1.0): 27,
 ('week', 1.0): 72,
 ('lambja', 1.0): 1,
 ('hey', 1.0): 60,
 ('jame', 1.0): 7,
 ('odd', 1.0): 2,
 ('plea', 1.0): 83,
 ('call', 1.0): 27,
 ('contact', 1.0): 4,
 ('centr', 1.0): 1,
 ('abl', 1.0): 6,
 ('assist', 1.0): 1,
 ('mani', 1.0): 28,
 ('thank', 1.0): 520,
 ('despiteoffici', 1.0): 1,
 ('listen', 1.0): 14,
 ('last', 1.0): 39,
 ('night', 1.0): 57,
 ('bleed', 1.0): 2,
 ('amaz', 1.0): 40,
 ('track', 1.0): 5,
 ('scotland', 1.0): 2,
 ('side', 1.0): 14,
 ('congrat', 1.0): 15,
 ('yeaaaah', 1.0): 1,
 ('yippppi', 1.0): 1,
 ('accnt', 1.0): 2,
 ('verifi', 1.0): 2,
 ('rqst', 1.0): 1,
 ('succeed', 1.0): 1,
 ('got', 1.0): 61,
 ('blue', 1.0): 8,
 ('tick', 1.0): 1,
 ('mark', 1.0): 2,
 ('fb', 1.0): 4,
 ('profil', 1.0): 2,
 ('day', 1.0): 184,
 ('bhaktisbant', 1.0): 16,
 ('pallaviruhail', 1.0): 8,
 ('one', 1.0): 92,
 ('i

In [31]:
def lookup(freqs,word,n):
  if (word,n) in freqs.keys():
    return freqs[(word,n)]
  return 0
lookup(freqs,'town',1)

4

In [34]:
def train_naive_bayes(freqs,train_x,train_y):
  log_likelihood={}
  logprior=0

  vocab=set([pair[0] for pair in freqs.keys()])
  v=len(vocab)

  Npos=Nneg=0
  for pair in freqs.keys():
    if pair[1]>0:
      Npos+=freqs[pair]
    else:
      Nneg+=freqs[pair]

  D=len(train_y)
  Dpos=np.sum(train_y)
  Dneg=D-Dpos
  logprior=np.log(Dpos)-np.log(Dneg)

  for word in vocab:
    freq_pos=lookup(freqs,word,1)
    freq_neg=lookup(freqs,word,0)

    pwpos=(freq_pos+1)/(Npos+v)
    pwneg=(freq_neg+1)/(Nneg+v)

    log_likelihood[word]=np.log(pwpos/pwneg)

  return logprior,log_likelihood


In [37]:
logprior,log_likelihood=train_naive_bayes(freqs,train_x,train_y)
print(log_likelihood)

{'thirdwheel': -0.7477690375385088, 'creed': -0.7477690375385088, 'fineanddandi': 0.6385253235813819, 'cabeilovato': -0.7477690375385088, 'dotacapitalist': -0.7477690375385088, 'sunnyclaribel': 0.6385253235813819, 'zzzz': -0.7477690375385088, 'arriebar': 0.6385253235813819, 'melonhairi': -0.7477690375385088, 'goe': -0.05462185697856331, 'keithrparson': 0.6385253235813819, 'blackreign': 0.6385253235813819, 'choma': -0.7477690375385088, 'periscop': -0.7477690375385088, 'otradaili': 0.6385253235813819, 'ada': 0.6385253235813819, 'heiya': 1.0439904316895463, 'mcjen': 0.6385253235813819, 'nechantya': 0.6385253235813819, 'edward': -0.05462185697856343, 'tearout': 0.6385253235813819, 'thedamarisduran': 0.6385253235813819, 'innoc': -0.7477690375385088, 'wolv': 1.0439904316895463, 'fluent': -0.7477690375385088, 'nrltigersroost': -0.7477690375385088, 'mxszwxrx': -0.7477690375385088, 'heyitsplat': -0.7477690375385088, 'nowse': 0.6385253235813819, 'penyfan': -0.7477690375385088, 'newyork': 0.63852

In [38]:
def naive_bayes_predict(tweet,logprior,log_likelihood):
  tweet=process_tweet(tweet)
  result=0
  for word in tweet:
    result+=log_likelihood[word]
  result+=logprior
  return result

In [93]:
def predict(t):
  ans=naive_bayes_predict(t,logprior,log_likelihood)
  if (ans>0 and ans<0.5):
    print('cool')
  elif (ans>0.5 and ans<2):
    print('Yay great')
  elif (ans>2 and ans<3):
    print('Fantastic')
  elif (ans<0 and ans>-1):
    print('sad')
  elif (ans<-1 and ans>-2):
    print('shittss')
  elif (ans>2):
    print('Whoaa lessgoo pawryyy')
  elif (ans<-2):
    print('omg i hope it gets better')

In [120]:
t='i am birthdaygirl'
predict(t)

Yay great
